### install dependencies
- if there's a pyproject.toml file:
uv sync
### or add one by one - I'm using google genai
uv add langchain-core
uv add langchain-google-genai
uv add langchain-community
uv add ipykernel
uv add pydantic
### have your api key in a .env file
and add it to .gitignore

In [ ]:
# get the apy key
import os
import getpass

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass.getpass(
    "Enter your Google API key: ")

#print(f"Google API key set successfully: {os.environ['GOOGLE_API_KEY']} ")

In [ ]:
#genai_model = "gemini-2.0-flash"
#genai_model = "gemini-2.5-flash-preview-05-20"
genai_model = "gemini-2.5-pro-preview-05-06"

### check the ChatGoogleGenerativeAI documentation
https://python.langchain.com/docs/integrations/chat/google_generative_ai/

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model=genai_model,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

creative_llm = ChatGoogleGenerativeAI(
    model=genai_model,
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

### prompts
- check the documentation https://python.langchain.com/api_reference/core/prompts.html
- we'll be using
-- SystemMessagePromptTemplate https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.SystemMessagePromptTemplate.html
-- HumanMessagePromptTemplate https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.HumanMessagePromptTemplate.html

In [9]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# define the system prompt
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a norwegian language teacher that answers questions about the Norwegian language. "
    "You are knowledgeable about its features, capabilities, and how to use it effectively."
    "You should provide clear and concise explanations, examples, and practical advice to help users understand the language better."
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """
    Your task is to answer the user's 
    ---
    {question}
    ---
    about the Norwegian language.
    Provide a detailed and informative response that addresses the user's query.
    If the user asks for examples, provide relevant examples in Norwegian with translations.
    If the user asks for practical advice, give actionable tips that can help them learn or use the language effectively.
    If the user asks for cultural context, provide insights into how the language is used in Norwegian culture.
    """,
    input_variables=["question"]
)

In [16]:
question = "What does the word 'ei' mean in Norwegian? Give me a grammar explanation and examples of real usage."

### create the chat prompt from the system prompt and user prompt
- we'll use
-- ChatPromptTemplate https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

In [10]:
from langchain.prompts import ChatPromptTemplate
first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [13]:
chain_one = (
    {"question": lambda x: x["question"]}
    | first_prompt | llm
    | {"answer": lambda x: x.content}
)

In [17]:
norwegian_answer_msg = chain_one.invoke({"question": question})
norwegian_answer_msg

{'answer': 'Hei! "Ei" is a versatile little word in Norwegian, and its meaning depends on the context. Here\'s a breakdown:\n\n**1. "Ei" as the feminine indefinite article:**\n\n*   **Grammar:** In Norwegian, nouns have grammatical gender: masculine, feminine, and neuter. "Ei" is the indefinite article used before feminine nouns in the singular. It\'s equivalent to "a" or "an" in English when referring to a feminine noun.\n\n*   **Usage:** It indicates that you\'re talking about *any* one of something, not a specific one.\n\n*   **Examples:**\n\n    *   "Jeg ser **ei** jente." (I see **a** girl.) - "Jente" (girl) is feminine.\n    *   "Hun har **ei** bok." (She has **a** book.) - "Bok" (book) is feminine.\n    *   "Vil du ha **ei** kake?" (Do you want **a** cake?) - "Kake" (cake) is feminine.\n\n*   **Important Note:** In Bokmål (the most common written standard), you can often use "en" (the masculine indefinite article) instead of "ei" for many feminine nouns. This is a result of hist

### with structured output using pydantic

In [21]:
from pydantic import BaseModel, Field

class NorwegianAnswer(BaseModel):
    answer: str = Field(..., description="The answer to the user's question about the Norwegian language.")
    examples: list[str] = Field(..., description="Examples of the word or expression in the question used in sentences.")
    grammar: str = Field(..., description="A brief explanation of the grammar rules related to the word or expression.")
    usage: list[str] = Field(..., description="Common phrases or sentences using the word or expression.")
    context: str = Field(..., description="The context in which the word or expression is typically used.")
    culture: str = Field(..., description="Cultural insights related to the word or expression in Norwegian language and culture.")

structured_llm = llm.with_structured_output(NorwegianAnswer)


In [25]:
chain_two = (
    {"question": lambda x: x["question"]}
    | first_prompt | structured_llm
    | {
        "answer": lambda x: x.answer,
        "examples": lambda x: x.examples,
        "grammar": lambda x: x.grammar,
        "usage": lambda x: x.usage,
        "context": lambda x: x.context,
        "culture": lambda x: x.culture
    }
)
structured_norwegian_answer = chain_two.invoke({"question": question})
structured_norwegian_answer

{'answer': "The word 'ei' in Norwegian is a feminine indefinite article, equivalent to 'a' or 'an' in English. It is used before nouns that are feminine in gender.",
 'examples': ['Ei jente (a girl)', 'Ei bok (a book)'],
 'grammar': "In Norwegian, nouns are divided into three genders: masculine, feminine, and neuter. The choice of 'ei' depends on the gender of the noun it precedes. 'Ei' is specifically for feminine nouns in the indefinite form.",
 'usage': ['Kan jeg låne ei bok? (Can I borrow a book?)',
  'Jeg ser ei jente. (I see a girl.)'],
 'context': 'Feminine nouns',
 'culture': 'Norwegian Bokmål has three grammatical genders: masculine, feminine, and neuter. However, in many dialects and in Nynorsk, the masculine and feminine genders are often merged into a common gender.'}